## Using CASE and WHEN
At times we might have to select values from multiple columns conditionally.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
val username = System.getProperty("user.name")

username = itv002461


itv002461

In [2]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Predefined Functions").
    master("yarn").
    getOrCreate

username = itv002461
spark = org.apache.spark.sql.SparkSession@df0e753


org.apache.spark.sql.SparkSession@df0e753

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

* We can use `CASE` and `WHEN` for that.
* Let us implement this conditional logic to come up with derived order_status.
  * If order_status is COMPLETE or CLOSED, set COMPLETED
  * If order_status have PENDING in it, then we will say PENDING
  * If order_status have PROCESSING or PAYMENT_REVIEW in it, then we will say PENDING
  * We will set all others as OTHER
* We can also have `ELSE` as part of `CASE` and `WHEN`.

In [3]:
%%sql

USE itv002461_retail

Waiting for a Spark session to start...

++
||
++
++



In [4]:
%%sql

SHOW tables

+----------------+--------------------+--...


+----------------+--------------------+-----------+
|        database|           tableName|isTemporary|
+----------------+--------------------+-----------+
|itv002461_retail|          categories|      false|
|itv002461_retail|           customers|      false|
|itv002461_retail|         departments|      false|
|itv002461_retail|                dual|      false|
|itv002461_retail|         order_items|      false|
|itv002461_retail|   order_items_stage|      false|
|itv002461_retail|              orders|      false|
|itv002461_retail|         orders_part|      false|
|itv002461_retail|orders_single_column|      false|
|itv002461_retail|            products|      false|
+----------------+--------------------+-----------+
only showing top 10 rows



In [5]:
%%sql

SELECT DISTINCT order_status FROM orders LIMIT 10

+---------------+
|   order_status|
+---------------+
|PENDING_PAYMENT|
|       COMPLETE|
|        ON_HOLD|
| PAYMENT_REVIEW|
|     PROCESSING|
|         CLOSED|
|SUSPECTED_FRAUD|
|        PENDING|
|       CANCELED|
+---------------+



In [7]:
%%sql

SELECT o.*,
    CASE WHEN order_status IN ('COMPLETE', 'CLOSED') THEN 'COMPLETED'
    END AS updated_order_status
FROM orders o
LIMIT 10

|       6|2013-07-25 ...


+--------+--------------------+-----------------+---------------+--------------------+
|order_id|          order_date|order_customer_id|   order_status|updated_order_status|
+--------+--------------------+-----------------+---------------+--------------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|           COMPLETED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|                null|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|           COMPLETED|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|           COMPLETED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|           COMPLETED|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|           COMPLETED|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|           COMPLETED|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|                null|
|       9|2013-07-25 00:00:...|            

In [8]:
%%sql

SELECT o.*,
    CASE WHEN order_status IN ('COMPLETE', 'CLOSED') THEN 'COMPLETED'
    ELSE order_status
    END AS updated_order_status
FROM orders o
LIMIT 10

|   34570|2014-02-23 ...


+--------+--------------------+-----------------+---------------+--------------------+
|order_id|          order_date|order_customer_id|   order_status|updated_order_status|
+--------+--------------------+-----------------+---------------+--------------------+
|   34565|2014-02-23 00:00:...|             8702|       COMPLETE|           COMPLETED|
|   34566|2014-02-23 00:00:...|             3066|PENDING_PAYMENT|     PENDING_PAYMENT|
|   34567|2014-02-23 00:00:...|             7314|SUSPECTED_FRAUD|     SUSPECTED_FRAUD|
|   34568|2014-02-23 00:00:...|             1271|       COMPLETE|           COMPLETED|
|   34569|2014-02-23 00:00:...|            11083|       COMPLETE|           COMPLETED|
|   34570|2014-02-23 00:00:...|             3159|         CLOSED|           COMPLETED|
|   34571|2014-02-23 00:00:...|             4551|         CLOSED|           COMPLETED|
|   34572|2014-02-23 00:00:...|             8135|        PENDING|             PENDING|
|   34573|2014-02-23 00:00:...|            

In [9]:
%%sql

SELECT o.*,
    CASE 
        WHEN order_status IN ('COMPLETE', 'CLOSED') THEN 'COMPLETED'
        WHEN order_status LIKE '%PENDING%' THEN 'PENDING'
        ELSE 'OTHER'
    END AS updated_order_status
FROM orders o
LIMIT 10

|       6|2013-07-25 ...


+--------+--------------------+-----------------+---------------+--------------------+
|order_id|          order_date|order_customer_id|   order_status|updated_order_status|
+--------+--------------------+-----------------+---------------+--------------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|           COMPLETED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|             PENDING|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|           COMPLETED|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|           COMPLETED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|           COMPLETED|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|           COMPLETED|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|           COMPLETED|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|               OTHER|
|       9|2013-07-25 00:00:...|            

In [10]:
%%sql

SELECT o.*,
    CASE 
        WHEN order_status IN ('COMPLETE', 'CLOSED') THEN 'COMPLETED'
        WHEN order_status LIKE '%PENDING%' OR order_status IN ('PROCESSING', 'PAYMENT_REVIEW')
            THEN 'PENDING'
        ELSE 'OTHER'
    END AS updated_order_status
FROM orders o
LIMIT 10

|   34570|2014-02-23 ...


+--------+--------------------+-----------------+---------------+--------------------+
|order_id|          order_date|order_customer_id|   order_status|updated_order_status|
+--------+--------------------+-----------------+---------------+--------------------+
|   34565|2014-02-23 00:00:...|             8702|       COMPLETE|           COMPLETED|
|   34566|2014-02-23 00:00:...|             3066|PENDING_PAYMENT|             PENDING|
|   34567|2014-02-23 00:00:...|             7314|SUSPECTED_FRAUD|               OTHER|
|   34568|2014-02-23 00:00:...|             1271|       COMPLETE|           COMPLETED|
|   34569|2014-02-23 00:00:...|            11083|       COMPLETE|           COMPLETED|
|   34570|2014-02-23 00:00:...|             3159|         CLOSED|           COMPLETED|
|   34571|2014-02-23 00:00:...|             4551|         CLOSED|           COMPLETED|
|   34572|2014-02-23 00:00:...|             8135|        PENDING|             PENDING|
|   34573|2014-02-23 00:00:...|            

In [11]:
%%sql

SELECT DISTINCT order_status,
    CASE 
        WHEN order_status IN ('COMPLETE', 'CLOSED') THEN 'COMPLETED'
        WHEN order_status LIKE '%PENDING%' OR order_status IN ('PROCESSING', 'PAYMENT_REVIEW')
            THEN 'PENDING'
        ELSE 'OTHER'
    END AS updated_order_status
FROM orders
ORDER BY updated_order_status

+---------------+--------------------+
|   order_status|updated_order_status|
+---------------+--------------------+
|       COMPLETE|           COMPLETED|
|         CLOSED|           COMPLETED|
|        ON_HOLD|               OTHER|
|SUSPECTED_FRAUD|               OTHER|
|       CANCELED|               OTHER|
|PENDING_PAYMENT|             PENDING|
| PAYMENT_REVIEW|             PENDING|
|     PROCESSING|             PENDING|
|        PENDING|             PENDING|
+---------------+--------------------+

